### Imports

In [181]:
import os
import numpy as np
import pandas as pd

### Inputs

In [182]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, 
medindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [183]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

- confidence (Confiança da Inferência do Modelo GNN): Eficácia do modelo gerado pelo GNN medido na atapa anterior.

In [184]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [185]:
# pesos podenrados
w_model, w_apmr, w_mfsr, w_idg = 0.4, 0.25, 0.25, 0.1
w_sum = w_model + w_apmr + w_mfsr + w_idg

In [186]:
confidence_model = np.where(inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [187]:
fused = (
    w_model * confidence_model +
    w_apmr * radnet_input_apmr_edge +
    w_mfsr * radnet_input_mfsr_edge +
    w_idg  * radnet_input_idg_global
) / w_sum

In [188]:
# classe final a partir do filtrado
cls = (fused >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, fused, 1.0 - fused)

### Apply

In [189]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}
for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-72    8.6   83     0.69   1        Bom      0.52      
-84    3.6   37     0.71   1        Bom      0.51      
-89    8.8   63     0.45   0        Ruim     0.62      
-36    3.0   22     0.20   1        Bom      0.91      
-58    4.8   53     0.75   1        Bom      0.51      
-57    1.5   22     0.37   1        Bom      0.86      
-83    7.4   89     0.37   0        Ruim     0.54      
-67    6.5   86     0.51   1        Bom      0.53      
-50    3.6   23     0.26   1        Bom      0.91      
-68    5.2   40     0.90   0        Ruim     0.68      
-87    9.6   65     0.66   0        Ruim     0.65      
-32    3.0   23     0.21   1        Bom      0.89      
-62    4.2   87     0.92   0        Ruim     0.64      
-67    3.2   36     0.32   1        Bom      0.68      
-65    3.0   36     0.22   1        Bom      0.91      
-37    2.5   31     0.29   1        Bom      0.9

### Save

In [190]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
